In [23]:
import pandas as pd
import numpy as np
from scipy.stats import norm, t
import matplotlib.pyplot as plt
from arch import arch_model
from arch.univariate.mean import ZeroMean
from arch.univariate.volatility import EWMAVariance, GARCH

## Portfolio

- SPY: €300 000
- AEX: €300 000
- RDSA: €10 000
- HSI: €90 000
- Bonds: €300 000

Total: €1 000 000

In [450]:
data = pd.read_csv('dataset.csv', index_col=0)
data.index = pd.to_datetime(data.index)

Z = np.log(data[['SPY', 'AEX', 'RDSA.AS', 'HSI', 'USDEUR', 'HKDEUR']])
Z['YIELDS'] = data['YIELDS'] / 100
X = z.diff()[1:]

W = np.array([0.3, 0.3, 0.01, 0.09, 0.3, 0.09, 0.3])

Cs = Z['YIELDS'][1:] / 250 * W[-1]
Bs = np.zeros(x.shape)
Bs[:,:] = W
Bs[:,-1] *= -np.arange(12, 0, -1/(250))[:len(x)]

principal = 1000000

## Helper functions

In [406]:
def linear_losses(x):
    return -Cs[:len(x)] - (Bs[len(x)] * x).sum(axis=1)

In [475]:
def normal_var(mean, std, alpha):
    return mean + std * norm.ppf(alpha)

def normal_es(mean, std, alpha):
    return mean + std * norm.pdf(norm.ppf(alpha)) / (1 - alpha)

def t_var(v, mean, std, alpha):
    t_std = std / np.sqrt(v / (v-2))
    return mean + std * t.ppf(alpha, v)

def t_es(v, mean, std, alpha):
    t_std = std / np.sqrt(v / (v-2))
    return mean + std * t.pdf(t.ppf(alpha, v), v) / (1 - alpha) * (v + t.ppf(alpha, v)**2) / (v-1)

def empirical_var(x, alpha):
    return np.quantile(np.sort(x), alpha, interpolation='linear')
    
def empirical_es(x, alpha):
    var = empirical_var(x, alpha)
    return x[x >= var].mean()

In [494]:
# def var_cov(x):
#     x_prime = x.copy()
#     x_prime.values[:,:] = x.mean()
#     c = C[:len(x)]
#     b = B[:len(x), :]

#     mean = - c - (b * x_prime).sum(axis=1)
#     std = np.sqrt((b.dot(x.cov()) * b).sum(axis=1))
#     return mean, std
def vc(x, mean=None, cov=None):
    c = Cs[len(x)]
    b = Bs[len(x), :]
    mu = mean if mean is not None else x.mean()
    sigma = cov if cov is not None else x.cov()

    mean = - c - (b * mu).sum()
    std = np.sqrt(b.dot(sigma).dot(b))
    return mean, std

In [496]:
mean, std = vc(x[:250])
normal_var(mean, std, 0.975)

0.012037053920784718

In [497]:
def ccc(x):
    p = x.corr()
    factor_models = [arch_model(x[column], mean='zero', p=1, q=1, rescale=True) for column in x.columns]
    results = [model.fit(update_freq=0) for model in factor_models]
    scales = [result.scale for result in results]
    forecasts = [result.forecast() for result in results]

    means = np.array([forecast.mean.values[-1, 0] for forecast in forecasts])
    stds = np.array([np.sqrt(forecast.variance.values[-1, 0]) for forecast in forecasts])
    
    means /= scales
    stds /= scales
    diag = np.diag(stds)
    cov = diag.dot(p).dot(diag)
    return vc(x, means, cov)

In [498]:
mean, cov = ccc(X[:250])
normal_var(mean, std, 0.975)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 356.35698344355586
            Iterations: 7
            Function evaluations: 45
            Gradient evaluations: 7
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 376.9837646581177
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 366.9377374911194
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 346.3582258275492
            Iterations: 6
            Function evaluations: 38
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 253.60926953043122
            Iterations: 15
            Function evaluat

0.012346334396850907

In [499]:
def fhs(x):
    factor_models = [ZeroMean(x[column], volatility=EWMAVariance(), rescale=True) for column in x.columns]
    results = [model.fit(update_freq=0) for model in factor_models]
    scales = [result.scale for result in results]
    forecasts = [result.forecast(start=0) for result in results]
    
    means = [forecast.mean for forecast in forecasts]
    stds = [np.sqrt(forecast.variance) for forecast in forecasts]
    
    means = pd.concat([mean.rename(columns={'h.1': column}) for mean, column in zip(means, x.columns)], axis=1)
    stds = pd.concat([std.rename(columns={'h.1': column}) for std, column in zip(stds, x.columns)], axis=1)
    
    means /= scales
    stds /= scales
    
    next_means = means.iloc[-1,:]
    next_stds = stds.iloc[-1,:]
    
    residuals = (x - means.shift().values) / stds.shift().values
    return linear_losses(next_means + next_stds * residuals)

In [500]:
simulated_losses = fhs(X[:250])
empirical_var(simulated_losses, 0.99)

0.009814570136581106